In [1]:
library(MASS)
library(nnet)
library(caret)

Warning message:
"package 'nnet' was built under R version 3.6.2"Warning message:
"package 'caret' was built under R version 3.6.2"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.2"

# Load Data and Final Clean

In [2]:
df <- read.csv(file="cleaned_data_02171.csv",sep=",",header=TRUE)
df = df[,-1]
colnames(df)
dim(df)

[1] "X1st_Term_Funding"    "X2nd_Term_Funding"    "X3rd_Term_Funding"   
 [4] "X4th_Term_Funding"    "Start_year"           "Start_age"           
 [7] "College"              "Age"                  "Degree"              
[10] "Gender"               "funding_indicator"    "ID"                  
[13] "Citizenship"          "Major_id"             "Major"               
[16] "net_cost"             "Program"              "Prep.Level"          
[19] "Total.Charges"        "Total.Funding.Amount" "Y"                   
[22] "PC1"                  "PC2"                  "PC3"                 
[25] "Term.1.GPA"           "Term.2.GPA"           "Term.3.GPA"          
[28] "Term.4.GPA"           "Term.5.GPA"           "Term.6.GPA"          
[31] "Term.7.GPA"           "Term.8.GPA"           "gap_indicator"       
[34] "GPA_trend"            "Valid.term"           "Average_GPA"         
[37] "academic_period"      "s_College"            "s_Citizenship"

[1] 2470   39

In [3]:
unique(df$Citizenship)

[1] White                        Black or African American   
 [3] Asian                        Asia                        
 [5] Hispanic or Latino           China                       
 [7] Africa                       Unknown                     
 [9] India                        Europe                      
[11] Other International Students
11 Levels: Africa Asia Asian Black or African American China ... White

Scale and center some variables:

In [4]:
mean_age = mean(df$Start_age)
sd_age = sd(df$Start_age)
df$s_start_age = (df$Start_age - mean_age)/sd_age
mean_net = mean(df$net_cost)
sd_net = sd(df$net_cost)
df$s_net_cost = (df$net_cost - mean_net)/sd_net
mean_year = mean(df$Start_year)
sd_year = sd(df$Start_year)
df$s_start_year = (df$Start_year - mean_year)/sd_year

In [5]:
av_net_cost = ifelse(df$Valid.term > 0, df$net_cost / df$Valid.term, df$net_cost)
av_mean_net = mean(av_net_cost)
av_sd_net = sd(av_net_cost)
df$avs_net_cost = (av_net_cost - av_mean_net)/av_sd_net

## Seperate by School

Different Schools

In [6]:
schools = levels(factor(df$College))

In [7]:
Harpur_df = df[df$College %in% schools[1:4],]
Management_df = df[df$College=="GD Management",]
Nursing_df = df[df$College=="GD Nursing",]
Watson_df = df[df$College=="GD Watson",]

In [8]:
unique(Harpur_df$Major)

[1] Chemistry                      Behavioral Neuroscience       
 [3] Cognitive Psychology           Physics                       
 [5] Mathematics                    Comparative Literature        
 [7] Translation Studies            Educational Theory and Practic
 [9] Philosophy Interpretation and  Biological Sciences           
[11] English                        Materials Science and Engineer
[13] History                        Anthropology                  
[15] Sociology                      Material Sci and Eng-Engineerg
[17] Clinical Psychology            Economics                     
[19] Geology                        Philosophy SPEL               
[21] Political Science              Community and Public Affairs  
[23] Art History                    Materials Science and Engr    
[25] Biological Sciences EEB        X Geology                     
[27] Art History Combined MA PhD    Mechanical Engineering        
[29] Mathematics/Economics         
45 Levels: Anthropology Art History ... XGerontolog Clin Nurse Special

In [9]:
harpur_sciences <- c("Chemistry", "Behavioral Neuroscience","Cognitive Psychology","Physics","Mathematics",
                     "Biological Sciences","Materials Science and Engineer","Material Sci and Eng-Engineerg",
                     "Clinical Psychology","Economics","Geology","Political Science","Materials Science and Engr",
                     "Biological Sciences EEB", "X Geology","Mathematics/Economics")
harpur_arts <- c("Comparative Literature","Translation Studies","Educational Theory and Practic",
                 "Philosophy Interpretation and", "English","History","Anthropology","Sociology","Philosophy SPEL",
                 "Community and Public Affairs","Art History","Art History Combined MA PhD")

In [10]:
Harpur_sciences = Harpur_df[Harpur_df$Major %in% harpur_sciences,]
Harpur_arts = Harpur_df[Harpur_df$Major %in% harpur_arts,]

In [11]:
Harpur_sciences <- within(Harpur_sciences, Citizenship <- relevel(Citizenship, ref = "White"))
Harpur_arts <- within(Harpur_arts, Citizenship <- relevel(Citizenship, ref = "White"))
Management_df <- within(Management_df, Citizenship <- relevel(Citizenship, ref = "White"))
Watson_df <- within(Watson_df, Citizenship <- relevel(Citizenship, ref = "White"))
Nursing_df <- within(Nursing_df, Citizenship <- relevel(Citizenship, ref = "White"))

In [12]:
dim(Harpur_sciences)[1]
dim(Harpur_arts)[1]
dim(Harpur_df)[1]

[1] 861

[1] 918

[1] 1780

In [13]:
a_n = dim(Harpur_sciences)[1]
a_1 = sum(Harpur_sciences$Y)
a_0 = a_n - a_1
b_n = dim(Harpur_arts)[1]
b_1 = sum(Harpur_arts$Y)
b_0 = b_n - b_1
c_n = dim(Management_df)[1]
c_1 = sum(Management_df$Y)
c_0 = c_n - c_1
d_n = dim(Nursing_df)[1]
d_1 = sum(Nursing_df$Y)
d_0 = d_n - d_1
e_n = dim(Watson_df)[1]
e_1 = sum(Watson_df$Y)
e_0 = e_n - e_1
School = c("Harpur sciences","Harpur_arts","SOM","Nursing","Watson")
sch_total = c(a_n,b_n,c_n,d_n,e_n)
succ_total = c(a_1,b_1,c_1,d_1,e_1)
unsucc_total = c(a_0,b_0,c_0,d_0,e_0)
data.frame("School"=School,"Total"=sch_total,"Successful Graduates"=succ_total,"Unsuccessful Graduates"=unsucc_total)

School,Total,Successful.Graduates,Unsuccessful.Graduates
Harpur sciences,861,542,319
Harpur_arts,918,534,384
SOM,58,38,20
Nursing,117,72,45
Watson,515,343,172


In [14]:
summary(Watson_df[which(Watson_df$Y==1), ]$Citizenship)

White                       Africa 
                          46                            6 
                        Asia                        Asian 
                          92                           16 
   Black or African American                        China 
                           7                           95 
                      Europe           Hispanic or Latino 
                           8                            6 
                       India Other International Students 
                          46                            2 
                     Unknown 
                          19

In [15]:
df1 = rbind(Harpur_sciences, Harpur_arts, Management_df, Watson_df, Nursing_df)
df1

,X1st_Term_Funding,X2nd_Term_Funding,X3rd_Term_Funding,X4th_Term_Funding,Start_year,Start_age,College,Age,Degree,Gender,...,GPA_trend,Valid.term,Average_GPA,academic_period,s_College,s_Citizenship,s_start_age,s_net_cost,s_start_year,avs_net_cost
1,4685,4935,4110.0,432.0,2013,22.3,GD Harpur,29,Doctor of Philosophy,Male,...,0.06,3,3.79,4.6,GD Harpur,White,-0.758619253,-0.826374490,0.85901134,-0.65494776
3,4685,4685,3699.0,3699.0,2012,22.0,GD Harpur,29,Doctor of Philosophy,Female,...,-0.08,6,3.14,5.4,GD Harpur,White,-0.798100147,-0.732387052,0.64687796,-0.68741906
4,4685,4685,3699.0,3699.0,2012,23.0,GD Harpur,30,Doctor of Philosophy,Female,...,0.00,6,3.98,4.4,GD Harpur,White,-0.666497166,-0.731452053,0.64687796,-0.68693362
11,3699,3699,432.0,432.0,2013,23.0,GD Harpur,29,Doctor of Philosophy,Male,...,0.00,2,3.70,5.0,GD Harpur,White,-0.666497166,1.846713327,0.85901134,3.58982146
12,8325,8325,925.0,925.0,2013,23.0,GD Harpur,29,Doctor of Philosophy,Female,...,0.03,6,3.96,4.4,GD Harpur,China,-0.666497166,-0.664944044,0.85901134,-0.65240349
14,0,0,3141.0,3141.0,2008,22.0,GD Harpur,33,Doctor of Philosophy,Female,...,0.08,6,3.69,5.4,GD Harpur,White,-0.798100147,0.115601102,-0.20165553,-0.24715426
23,0,0,3141.0,3141.0,2008,33.0,GD Harpur,44,Doctor of Philosophy,Male,...,-0.07,4,3.83,5.0,GD Harpur,White,0.649532636,-0.429510596,-0.20165553,-0.38651183
26,0,0,4968.0,4968.0,2008,22.0,GD Harpur,33,Doctor of Philosophy,Female,...,0.13,4,3.84,4.4,GD Harpur,White,-0.798100147,-0.170550395,-0.20165553,-0.18483851
29,4968,6072,5166.0,5683.2,2009,24.0,GD Grad School,34,Doctor of Philosophy,Male,...,0.25,3,3.68,6.0,GD Non-Harpur,White,-0.534894186,-0.245971116,0.01047784,-0.05227148
32,0,0,0.0,4968.0,2008,23.3,GD Harpur,35,Doctor of Philosophy,Female,...,0.38,5,3.34,5.0,GD Harpur,White,-0.627016272,0.214431648,-0.20165553,-0.07151431


# Harpur Sciences Analysis

Won't use Degree column since it does not accurately show the student's major

<b> Logistic regression model:

In [16]:
log_mod_h_s = glm(Y~s_start_age+factor(Gender)+factor(funding_indicator)+factor(Citizenship)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year,
                data=Harpur_sciences,family="binomial")
summary(log_mod_h_s)


Call:
glm(formula = Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year, family = "binomial", data = Harpur_sciences)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.2424  -0.9398   0.5583   0.8578   2.1477  

Coefficients:
                                                 Estimate Std. Error z value
(Intercept)                                     -0.470269   0.260006  -1.809
s_start_age                                      0.141085   0.148570   0.950
factor(Gender)Male                               0.003381   0.164442   0.021
factor(funding_indicator)1                       1.299581   0.255741   5.082
factor(Citizenship)Africa                        0.988544   0.475913   2.077
factor(Citizenship)Asia                          0.313186   0.354419   0.884
factor(Citizenship)Asian                        -0.334214   0.426055  -0.784
factor(Citi

<b> Significant Variables of logistic regression model: 

In [17]:
round(summary(log_mod_h_s)$coeff[which(summary(log_mod_h_s)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,1.29958,0.00000
factor(Citizenship)Africa,0.98854,0.03779
factor(Citizenship)China,0.61341,0.01006
GPA_trend,1.07589,0.01117
s_start_year,-1.34718,0.00000


Things we need
-  Run AIC/BIC stepwise model selection for Harpur and summarize results
-  Repeat for the other schools

<b> Both AIC:

In [18]:
library('MASS')
log_mod_h_s_aic = stepAIC(log_mod_h_s, direction = 'both')

Start:  AIC=995.15
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance     AIC
- factor(Gender)             1   953.15  993.15
- PC3                        1   953.25  993.25
- PC1                        1   953.32  993.32
- PC2                        1   953.83  993.83
- factor(Citizenship)       10   971.86  993.86
- s_start_age                1   954.06  994.06
<none>                           953.15  995.15
- avs_net_cost               1   955.34  995.34
- factor(gap_indicator)      1   956.83  996.83
- GPA_trend                  1   960.25 1000.25
- factor(funding_indicator)  1   979.68 1019.68
- s_start_year               1  1077.03 1117.03

Step:  AIC=993.15
Y ~ s_start_age + factor(funding_indicator) + factor(Citizenship) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year


<b> Significant variables both AIC selects:

In [19]:
round(summary(log_mod_h_s_aic)$coeff[which(summary(log_mod_h_s_aic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,1.21272,0.00000
GPA_trend,0.96259,0.01774
s_start_year,-1.34133,0.00000


<b> Both BIC

In [20]:
log_mod_h_s_bic = stepAIC(log_mod_h_s, direction = "both", k = log(a_n))

Start:  AIC=1095.07
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- factor(Citizenship)       10   971.86 1046.2
- factor(Gender)             1   953.15 1088.3
- PC3                        1   953.25 1088.4
- PC1                        1   953.32 1088.5
- PC2                        1   953.83 1089.0
- s_start_age                1   954.06 1089.2
- avs_net_cost               1   955.34 1090.5
- factor(gap_indicator)      1   956.83 1092.0
<none>                           953.15 1095.1
- GPA_trend                  1   960.25 1095.4
- factor(funding_indicator)  1   979.68 1114.8
- s_start_year               1  1077.03 1212.2

Step:  AIC=1046.2
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                

<b> Significant variables both BIC selects:

In [21]:
round(summary(log_mod_h_s_bic)$coeff[which(summary(log_mod_h_s_bic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,1.18085,0
s_start_year,-1.38659,0


# Harpur Arts Analysis

<b> Logistic regression model:

In [22]:
log_mod_h_a = glm(Y~s_start_age+factor(Gender)+factor(funding_indicator)+factor(Citizenship)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year,
                data=Harpur_arts,family="binomial")
summary(log_mod_h_a)


Call:
glm(formula = Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year, family = "binomial", data = Harpur_arts)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1179  -1.1715   0.7647   1.0103   1.6159  

Coefficients:
                                                 Estimate Std. Error z value
(Intercept)                                     -0.343307   0.166521  -2.062
s_start_age                                      0.106362   0.070265   1.514
factor(Gender)Male                              -0.007463   0.143869  -0.052
factor(funding_indicator)1                       0.700065   0.172111   4.068
factor(Citizenship)Africa                       -0.389627   0.529915  -0.735
factor(Citizenship)Asia                          0.692405   0.236852   2.923
factor(Citizenship)Asian                        -0.383150   0.384344  -0.997
factor(Citizens

<b> Significant Variables of logistic regression model: 

In [23]:
round(summary(log_mod_h_a)$coeff[which(summary(log_mod_h_a)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-0.34331,0.03924
factor(funding_indicator)1,0.70006,0.00005
factor(Citizenship)Asia,0.69241,0.00346
s_start_year,-0.51795,0.00000


<b> Both AIC:

In [24]:
log_mod_h_a_aic = stepAIC(log_mod_h_a, direction = 'both')

Start:  AIC=1212.21
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- factor(Citizenship)       10   1184.7 1206.7
- factor(Gender)             1   1170.2 1210.2
- PC2                        1   1170.2 1210.2
- avs_net_cost               1   1170.2 1210.2
- factor(gap_indicator)      1   1171.0 1211.0
- PC3                        1   1171.2 1211.2
- GPA_trend                  1   1171.9 1211.9
<none>                           1170.2 1212.2
- PC1                        1   1172.3 1212.3
- s_start_age                1   1172.5 1212.5
- factor(funding_indicator)  1   1187.2 1227.2
- s_start_year               1   1220.3 1260.3

Step:  AIC=1206.66
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

               

<b> Significant variables both AIC selects:

In [25]:
round(summary(log_mod_h_a_aic)$coeff[which(summary(log_mod_h_a_aic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,0.63680,8e-05
s_start_year,-0.48729,0e+00


<b> Both BIC

In [26]:
log_mod_h_a_bic = stepAIC(log_mod_h_a, direction = "both", k = log(b_n))

Start:  AIC=1313.48
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- factor(Citizenship)       10   1184.7 1259.7
- factor(Gender)             1   1170.2 1306.7
- PC2                        1   1170.2 1306.7
- avs_net_cost               1   1170.2 1306.7
- factor(gap_indicator)      1   1171.0 1307.5
- PC3                        1   1171.2 1307.7
- GPA_trend                  1   1171.9 1308.4
- PC1                        1   1172.3 1308.7
- s_start_age                1   1172.5 1309.0
<none>                           1170.2 1313.5
- factor(funding_indicator)  1   1187.2 1323.7
- s_start_year               1   1220.3 1356.8

Step:  AIC=1259.7
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                

<b> Significant variables both BIC selects:

In [27]:
round(summary(log_mod_h_a_bic)$coeff[which(summary(log_mod_h_a_bic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,0.61318,0.00011
s_start_year,-0.49610,0.00000


# Management Analysis

<b> Logistic regression model:

In [28]:
log_mod_m = glm(Y~s_start_age+factor(Gender)+factor(Citizenship)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year,
                data=Management_df,family="binomial")
summary(log_mod_m)


Call:
glm(formula = Y ~ s_start_age + factor(Gender) + factor(Citizenship) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year, family = "binomial", data = Management_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1948  -0.8325   0.4263   0.8166   1.5902  

Coefficients:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                              1.45588    1.20046   1.213    0.225
s_start_age                              0.38243    0.73882   0.518    0.605
factor(Gender)Male                      -0.85210    0.75490  -1.129    0.259
factor(Citizenship)Asia                  0.29362    1.12511   0.261    0.794
factor(Citizenship)Asian                 1.57017    1.71928   0.913    0.361
factor(Citizenship)China                 0.73942    1.45250   0.509    0.611
factor(Citizenship)Europe               17.04912 3956.18052   0.004    0.997
factor(Citizenship)Hispanic or Latino    

<b> Significant Variables of logistic regression model: 

In [29]:
round(summary(log_mod_m)$coeff[which(summary(log_mod_m)$coef[,4] <= .05),c(1,4)],5)

Estimate,Pr(>|z|)


<b> Both AIC:

In [30]:
log_mod_m_aic= stepAIC(log_mod_m, direction = 'both')

Start:  AIC=90.4
Y ~ s_start_age + factor(Gender) + factor(Citizenship) + avs_net_cost + 
    PC1 + PC2 + PC3 + factor(gap_indicator) + GPA_trend + s_start_year

                        Df Deviance    AIC
- factor(Citizenship)    7   58.210 78.210
- s_start_year           1   56.404 88.404
- PC3                    1   56.453 88.453
- s_start_age            1   56.672 88.672
- PC2                    1   56.753 88.753
- factor(Gender)         1   57.696 89.696
- PC1                    1   57.867 89.867
- GPA_trend              1   57.937 89.937
- avs_net_cost           1   58.312 90.312
<none>                       56.399 90.399
- factor(gap_indicator)  1   60.177 92.177

Step:  AIC=78.21
Y ~ s_start_age + factor(Gender) + avs_net_cost + PC1 + PC2 + 
    PC3 + factor(gap_indicator) + GPA_trend + s_start_year

                        Df Deviance    AIC
- s_start_age            1   58.227 76.227
- PC3                    1   58.418 76.418
- s_start_year           1   58.423 76.423
- PC2    

<b> Significant variables both AIC selects:

In [31]:
summary(log_mod_m_aic)


Call:
glm(formula = Y ~ avs_net_cost + PC1 + factor(gap_indicator) + 
    GPA_trend, family = "binomial", data = Management_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4385  -0.8551   0.6446   0.8944   1.7533  

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)  
(Intercept)               1.388      0.645   2.152   0.0314 *
avs_net_cost              1.703      1.296   1.315   0.1886  
PC1                       2.163      1.202   1.800   0.0719 .
factor(gap_indicator)1   16.918   1692.845   0.010   0.9920  
GPA_trend                 2.840      1.716   1.656   0.0978 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 74.726  on 57  degrees of freedom
Residual deviance: 60.169  on 53  degrees of freedom
AIC: 70.169

Number of Fisher Scoring iterations: 16


<b> Both BIC:

In [32]:
log_mod_m_bic = stepAIC(log_mod_m, direction = "both", k = log(c_n))

Start:  AIC=125.43
Y ~ s_start_age + factor(Gender) + factor(Citizenship) + avs_net_cost + 
    PC1 + PC2 + PC3 + factor(gap_indicator) + GPA_trend + s_start_year

                        Df Deviance     AIC
- factor(Citizenship)    7   58.210  98.814
- s_start_year           1   56.404 121.371
- PC3                    1   56.453 121.420
- s_start_age            1   56.672 121.640
- PC2                    1   56.753 121.720
- factor(Gender)         1   57.696 122.663
- PC1                    1   57.867 122.834
- GPA_trend              1   57.937 122.904
- avs_net_cost           1   58.312 123.279
- factor(gap_indicator)  1   60.177 125.144
<none>                       56.399 125.427

Step:  AIC=98.81
Y ~ s_start_age + factor(Gender) + avs_net_cost + PC1 + PC2 + 
    PC3 + factor(gap_indicator) + GPA_trend + s_start_year

                        Df Deviance     AIC
- s_start_age            1   58.227  94.771
- PC3                    1   58.418  94.962
- s_start_year           1   58.423

<b> Significant variables both BIC selects:

In [33]:
round(summary(log_mod_m_bic)$coeff[which(summary(log_mod_m_bic)$coef[,4] <= .05),c(1,4)],5)
summary(log_mod_m_bic)

,Estimate,Pr(>|z|)
(Intercept),0.63510,0.03684
PC1,2.37914,0.03670



Call:
glm(formula = Y ~ PC1 + factor(gap_indicator), family = "binomial", 
    data = Management_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6918  -1.1190   0.7180   0.9222   1.9057  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)  
(Intercept)               0.6351     0.3042   2.088   0.0368 *
PC1                       2.3791     1.1389   2.089   0.0367 *
factor(gap_indicator)1   17.0024  1719.3142   0.010   0.9921  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 74.726  on 57  degrees of freedom
Residual deviance: 64.973  on 55  degrees of freedom
AIC: 70.973

Number of Fisher Scoring iterations: 16


# Nursing Analysis

<b> Logistic regression model:

In [34]:
log_mod_n = glm(Y~s_start_age+factor(Gender)+factor(funding_indicator)+factor(Citizenship)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year,
                data=Nursing_df,family="binomial")
summary(log_mod_n)


Call:
glm(formula = Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year, family = "binomial", data = Nursing_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2197  -0.9799   0.3417   0.9461   1.5818  

Coefficients:
                                              Estimate Std. Error z value
(Intercept)                                     0.7180     0.5011   1.433
s_start_age                                    -0.2134     0.1670  -1.278
factor(Gender)Male                              0.2041     0.7147   0.286
factor(funding_indicator)1                      0.4484     0.5059   0.886
factor(Citizenship)Africa                      16.6571  2224.0360   0.007
factor(Citizenship)Asia                         0.1680     1.3517   0.124
factor(Citizenship)Asian                        0.9042     0.9390   0.963
factor(Citizenship)Black or African Amer

<b> Significant Variables of logistic regression model: 

In [35]:
round(summary(log_mod_n)$coeff[which(summary(log_mod_n)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
PC3,-1.39774,0.04619
s_start_year,-0.77483,0.01200


<b> Both AIC:

In [36]:
log_mod_n_aic= stepAIC(log_mod_n, direction = 'both')

Start:  AIC=159.9
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- avs_net_cost               1   123.92 157.92
- GPA_trend                  1   123.98 157.98
- factor(Gender)             1   123.98 157.98
- factor(gap_indicator)      1   124.65 158.65
- factor(funding_indicator)  1   124.70 158.70
- factor(Citizenship)        7   137.48 159.48
- PC2                        1   125.49 159.49
- s_start_age                1   125.58 159.58
<none>                           123.90 159.90
- PC1                        1   127.22 161.22
- PC3                        1   127.93 161.93
- s_start_year               1   132.68 166.68

Step:  AIC=157.92
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

           

<b> Significant variables both AIC selects:

In [37]:
round(summary(log_mod_n_aic)$coeff[which(summary(log_mod_n_aic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),1.01133,0.00927
PC3,-1.56605,0.01369
s_start_year,-0.75705,0.01073


<b> Both BIC:

In [38]:
log_mod_n_bic = stepAIC(log_mod_n, direction = "both", k = log(d_n))

Start:  AIC=209.62
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- factor(Citizenship)        7   137.48 189.86
- avs_net_cost               1   123.92 204.87
- GPA_trend                  1   123.98 204.94
- factor(Gender)             1   123.98 204.94
- factor(gap_indicator)      1   124.65 205.60
- factor(funding_indicator)  1   124.70 205.66
- PC2                        1   125.49 206.45
- s_start_age                1   125.58 206.53
- PC1                        1   127.22 208.18
- PC3                        1   127.93 208.88
<none>                           123.90 209.62
- s_start_year               1   132.68 213.63

Step:  AIC=189.86
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                 

<b> Significant variables both BIC selects:

In [39]:
round(summary(log_mod_n_bic)$coeff[which(summary(log_mod_n_bic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),0.73862,0.00187
s_start_year,-0.62592,0.01300


# Watson Analysis 

<b> Logistic regression model:

In [40]:
log_mod_w = glm(Y~s_start_age+factor(Gender)+factor(funding_indicator)+factor(Citizenship)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year,
                data=Watson_df,family="binomial")
summary(log_mod_w)


Call:
glm(formula = Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year, family = "binomial", data = Watson_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8526  -0.8246   0.5108   0.7890   1.8463  

Coefficients:
                                                 Estimate Std. Error z value
(Intercept)                                      -1.01412    0.39744  -2.552
s_start_age                                      -0.06420    0.14321  -0.448
factor(Gender)Male                                0.32809    0.27567   1.190
factor(funding_indicator)1                        1.93993    0.30301   6.402
factor(Citizenship)Africa                        15.18479  533.86250   0.028
factor(Citizenship)Asia                           0.73354    0.37279   1.968
factor(Citizenship)Asian                         -0.06867    0.51749  -0.133
factor(Citizenshi

<b> Significant Variables of logistic regression model: 

In [41]:
round(summary(log_mod_w)$coeff[which(summary(log_mod_w)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-1.01412,0.01072
factor(funding_indicator)1,1.93993,0.00000
factor(Citizenship)Asia,0.73354,0.04910
avs_net_cost,0.37210,0.01321
PC3,-0.42651,0.01220
s_start_year,-0.98997,0.00000


<b> Both AIC:

In [42]:
log_mod_w_aic= stepAIC(log_mod_w, direction = 'both')

Start:  AIC=561.18
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- factor(Citizenship)       10   535.85 557.85
- s_start_age                1   519.39 559.39
- factor(gap_indicator)      1   520.12 560.12
- PC2                        1   520.53 560.53
- factor(Gender)             1   520.59 560.59
<none>                           519.18 561.18
- GPA_trend                  1   521.91 561.91
- PC1                        1   522.00 562.00
- PC3                        1   526.60 566.60
- avs_net_cost               1   527.42 567.42
- s_start_year               1   561.33 601.33
- factor(funding_indicator)  1   564.70 604.70

Step:  AIC=557.85
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                 

<b> Significant variables both AIC selects:

In [43]:
round(summary(log_mod_w_aic)$coeff[which(summary(log_mod_w_aic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-0.54971,0.00760
factor(funding_indicator)1,2.01370,0.00000
avs_net_cost,0.36130,0.01750
PC1,-0.78742,0.01415
PC3,-0.45321,0.00415
s_start_year,-0.99717,0.00000


<b> Both BIC:

In [44]:
log_mod_w_bic = stepAIC(log_mod_w, direction = "both", k = log(e_n))

Start:  AIC=650.31
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    factor(Citizenship) + avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                            Df Deviance    AIC
- factor(Citizenship)       10   535.85 604.53
- s_start_age                1   519.39 644.27
- factor(gap_indicator)      1   520.12 645.00
- PC2                        1   520.53 645.41
- factor(Gender)             1   520.59 645.47
- GPA_trend                  1   521.91 646.80
- PC1                        1   522.00 646.89
<none>                           519.18 650.31
- PC3                        1   526.60 651.48
- avs_net_cost               1   527.42 652.30
- s_start_year               1   561.33 686.21
- factor(funding_indicator)  1   564.70 689.58

Step:  AIC=604.53
Y ~ s_start_age + factor(Gender) + factor(funding_indicator) + 
    avs_net_cost + PC1 + PC2 + PC3 + factor(gap_indicator) + 
    GPA_trend + s_start_year

                 

<b> Significant variables both BIC selects:

In [45]:
summary(log_mod_w_bic)


Call:
glm(formula = Y ~ factor(funding_indicator) + avs_net_cost + 
    s_start_year, family = "binomial", data = Watson_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8893  -0.9101   0.5567   0.8484   1.7719  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -0.5034     0.2001  -2.516   0.0119 *  
factor(funding_indicator)1   2.0988     0.2644   7.939 2.04e-15 ***
avs_net_cost                 0.3515     0.1548   2.270   0.0232 *  
s_start_year                -1.0004     0.1437  -6.963 3.34e-12 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 656.07  on 514  degrees of freedom
Residual deviance: 551.63  on 511  degrees of freedom
AIC: 559.63

Number of Fisher Scoring iterations: 4


In [46]:
round(summary(log_mod_w_bic)$coeff[which(summary(log_mod_w_bic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-0.50338,0.01188
factor(funding_indicator)1,2.09885,0.00000
avs_net_cost,0.35147,0.02320
s_start_year,-1.00040,0.00000


# Joint Analysis

<b> Logistic regression model:

In [47]:
int_chn = function(x) ifelse(x=='China',1,0)
int_afc = function(x) ifelse(x=='Africa',1,0)
int_asia = function(x) ifelse(x=='Asia',1,0)

In [48]:
df <- within(df, s_Citizenship <- relevel(s_Citizenship, ref = "White"))
df <- within(df, s_College <- relevel(s_College, ref = "GD Non-Harpur"))

In [49]:
log_mod_total = glm(Y~factor(funding_indicator)+factor(Citizenship)+
                avs_net_cost+PC1+PC3+GPA_trend+s_start_year+factor(s_College)+avs_net_cost:factor(s_College)+
                factor(s_College):factor(s_Citizenship)+factor(s_College):PC1+factor(s_College):PC3+
                factor(s_College):factor(funding_indicator)+factor(s_College):GPA_trend+s_start_year:factor(s_College),
                data=df,family="binomial")
summary(log_mod_total)


Call:
glm(formula = Y ~ factor(funding_indicator) + factor(Citizenship) + 
    avs_net_cost + PC1 + PC3 + GPA_trend + s_start_year + factor(s_College) + 
    avs_net_cost:factor(s_College) + factor(s_College):factor(s_Citizenship) + 
    factor(s_College):PC1 + factor(s_College):PC3 + factor(s_College):factor(funding_indicator) + 
    factor(s_College):GPA_trend + s_start_year:factor(s_College), 
    family = "binomial", data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7049  -1.0981   0.6572   0.9316   1.8365  

Coefficients: (3 not defined because of singularities)
                                                           Estimate Std. Error
(Intercept)                                                 0.61178    0.35975
factor(funding_indicator)1                                  1.27369    0.18564
factor(Citizenship)Asia                                    -0.09551    0.37139
factor(Citizenship)Asian                                   -0.99189    0.3942

<b> Significant Variables of logistic regression model:

In [50]:
round(summary(log_mod_total)$coeff[which(summary(log_mod_total)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,1.27369,0.00000
factor(Citizenship)Asian,-0.99189,0.01187
factor(Citizenship)Hispanic or Latino,-0.77489,0.04705
factor(Citizenship)Unknown,-0.78835,0.03215
factor(Citizenship)White,-0.70597,0.03489
avs_net_cost,0.37026,0.00163
PC3,-0.26165,0.00226
GPA_trend,1.30377,0.00884
s_start_year,-0.72536,0.00000
avs_net_cost:factor(s_College)GD Harpur,-0.35663,0.00541


<b> Both AIC:

In [51]:
log_mod_total_aic= stepAIC(log_mod_total, direction = 'both')

Start:  AIC=2996.08
Y ~ factor(funding_indicator) + factor(Citizenship) + avs_net_cost + 
    PC1 + PC3 + GPA_trend + s_start_year + factor(s_College) + 
    avs_net_cost:factor(s_College) + factor(s_College):factor(s_Citizenship) + 
    factor(s_College):PC1 + factor(s_College):PC3 + factor(s_College):factor(funding_indicator) + 
    factor(s_College):GPA_trend + s_start_year:factor(s_College)

                                              Df Deviance    AIC
- factor(Citizenship)                          7   2946.7 2986.7
- factor(s_College):factor(s_Citizenship)      3   2943.8 2991.8
- s_start_year:factor(s_College)               1   2942.5 2994.5
- GPA_trend:factor(s_College)                  1   2942.9 2994.9
<none>                                             2942.1 2996.1
- factor(funding_indicator):factor(s_College)  1   2944.2 2996.2
- PC1:factor(s_College)                        1   2944.8 2996.8
- PC3:factor(s_College)                        1   2947.9 2999.9
- avs_net_cost:f

<b> Significant variables both AIC selects:

In [52]:
round(summary(log_mod_total_aic)$coeff[which(summary(log_mod_total_aic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,1.32091,0.00000
avs_net_cost,0.37904,0.00135
PC3,-0.25247,0.00300
GPA_trend,0.92467,0.00061
s_start_year,-0.78581,0.00000
avs_net_cost:factor(s_College)GD Harpur,-0.37465,0.00356
factor(s_College)GD Harpur:factor(s_Citizenship)Africa,0.69759,0.03538
factor(s_College)GD Harpur:factor(s_Citizenship)Asia,0.60544,0.00157
PC3:factor(s_College)GD Harpur,0.23035,0.02040


<b> Both BIC:

In [53]:
log_mod_total_bic = stepAIC(log_mod_total, direction = "both", k = log(e_n))

Start:  AIC=3110.67
Y ~ factor(funding_indicator) + factor(Citizenship) + avs_net_cost + 
    PC1 + PC3 + GPA_trend + s_start_year + factor(s_College) + 
    avs_net_cost:factor(s_College) + factor(s_College):factor(s_Citizenship) + 
    factor(s_College):PC1 + factor(s_College):PC3 + factor(s_College):factor(funding_indicator) + 
    factor(s_College):GPA_trend + s_start_year:factor(s_College)

                                              Df Deviance    AIC
- factor(Citizenship)                          7   2946.7 3071.6
- factor(s_College):factor(s_Citizenship)      3   2943.8 3093.7
- s_start_year:factor(s_College)               1   2942.5 3104.9
- GPA_trend:factor(s_College)                  1   2942.9 3105.2
- factor(funding_indicator):factor(s_College)  1   2944.2 3106.6
- PC1:factor(s_College)                        1   2944.8 3107.1
- PC3:factor(s_College)                        1   2947.9 3110.3
<none>                                             2942.1 3110.7
- avs_net_cost:f


Step:  AIC=3031.26
Y ~ factor(funding_indicator) + avs_net_cost + GPA_trend + s_start_year + 
    factor(s_College) + avs_net_cost:factor(s_College) + factor(funding_indicator):factor(s_College)

                                              Df Deviance    AIC
<none>                                             2981.3 3031.3
- factor(funding_indicator):factor(s_College)  1   2989.0 3032.7
+ PC1                                          1   2979.2 3035.4
- avs_net_cost:factor(s_College)               1   2991.8 3035.5
+ PC3                                          1   2979.7 3035.9
+ GPA_trend:factor(s_College)                  1   2980.4 3036.6
+ s_start_year:factor(s_College)               1   2981.2 3037.4
- GPA_trend                                    1   2994.4 3038.1
+ factor(s_College):factor(s_Citizenship)      6   2958.8 3046.2
+ factor(Citizenship)                         10   2956.1 3068.5
- s_start_year                                 1   3194.6 3238.3


<b>Significant variables both BIC selects:

In [54]:
round(summary(log_mod_total_bic)$coeff[which(summary(log_mod_total_bic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
factor(funding_indicator)1,1.45200,0.00000
avs_net_cost,0.37658,0.00161
GPA_trend,0.92429,0.00049
s_start_year,-0.77343,0.00000
avs_net_cost:factor(s_College)GD Harpur,-0.38151,0.00322
factor(funding_indicator)1:factor(s_College)GD Harpur,-0.57854,0.00548


# Prediction

In [55]:
colnames(df)

[1] "X1st_Term_Funding"    "X2nd_Term_Funding"    "X3rd_Term_Funding"   
 [4] "X4th_Term_Funding"    "Start_year"           "Start_age"           
 [7] "College"              "Age"                  "Degree"              
[10] "Gender"               "funding_indicator"    "ID"                  
[13] "Citizenship"          "Major_id"             "Major"               
[16] "net_cost"             "Program"              "Prep.Level"          
[19] "Total.Charges"        "Total.Funding.Amount" "Y"                   
[22] "PC1"                  "PC2"                  "PC3"                 
[25] "Term.1.GPA"           "Term.2.GPA"           "Term.3.GPA"          
[28] "Term.4.GPA"           "Term.5.GPA"           "Term.6.GPA"          
[31] "Term.7.GPA"           "Term.8.GPA"           "gap_indicator"       
[34] "GPA_trend"            "Valid.term"           "Average_GPA"         
[37] "academic_period"      "s_College"            "s_Citizenship"       
[40] "s_start_age"          "s_net_cost"           "s_start_year"        
[43] "avs_net_cost"

In [56]:
colnames(df)[c(5,6,12,13,14,16,18,19,20,25:32)]

[1] "Start_year"           "Start_age"            "ID"                  
 [4] "Citizenship"          "Major_id"             "net_cost"            
 [7] "Prep.Level"           "Total.Charges"        "Total.Funding.Amount"
[10] "Term.1.GPA"           "Term.2.GPA"           "Term.3.GPA"          
[13] "Term.4.GPA"           "Term.5.GPA"           "Term.6.GPA"          
[16] "Term.7.GPA"           "Term.8.GPA"

In [57]:
df = df[,-c(5,6,12,13,14,16,18,19,20,25:32)]
colnames(df)

[1] "X1st_Term_Funding" "X2nd_Term_Funding" "X3rd_Term_Funding"
 [4] "X4th_Term_Funding" "College"           "Age"              
 [7] "Degree"            "Gender"            "funding_indicator"
[10] "Major"             "Program"           "Y"                
[13] "PC1"               "PC2"               "PC3"              
[16] "gap_indicator"     "GPA_trend"         "Valid.term"       
[19] "Average_GPA"       "academic_period"   "s_College"        
[22] "s_Citizenship"     "s_start_age"       "s_net_cost"       
[25] "s_start_year"      "avs_net_cost"

In [58]:
set.seed(123)
s1 = sample(nrow(df),0.5*nrow(df))
train = df[s1,]
non_train = df[-s1,]
s2 = sample(nrow(non_train),0.5*nrow(non_train))
test = non_train[s2,]
vld = non_train[-s2,]

train_y=train[,12]
train_x=train[,-12]
test_y=test[,12]
test_x=test[,-12]
dim(train)
dim(test)

[1] 1235   26

[1] 617  26

## LDA

In [59]:
mod_lda = lda(Y~s_start_age+factor(Gender)+factor(funding_indicator)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year
                + Valid.term + Average_GPA + academic_period + College + Age,
              data = as.data.frame(train))
lda_pred = predict(mod_lda, newdata = test)

In [60]:
(lda_acc = round(sum(lda_pred$class == test$Y)/length(lda_pred$class),4))

[1] 0.7618

## Neural Network

In [87]:
nnet_vld = NULL
for (i in seq(1,20,1)){
    mod_nnet = nnet(Y~s_start_age+factor(Gender)+factor(funding_indicator)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+Average_GPA+Valid.term,
                data = train, size  = i,trace = FALSE, maxit = 250)
    nn_pred = predict(mod_nnet, newdata = vld)
    nn_pred = as.numeric(nn_pred>=.5)
    nnet_vld = c(nnet_vld, sum(nn_pred == vld$Y)/length(nn_pred))
}
(tr_size = which.max(nnet_vld))

[1] 7

In [88]:
mod_nnet = nnet(Y~s_start_age+factor(Gender)+factor(funding_indicator)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+Average_GPA+Valid.term,
                data = train, size  = tr_size,trace = FALSE, maxit = 250)
nn_pred = as.numeric(predict(mod_nnet, newdata = test)>= .5)

In [89]:
(nnet_acc = round(sum(nn_pred == test$Y)/length(nn_pred),4))

[1] 0.7147

## Generalized Regression

In [90]:
mod_glm = glm(Y~s_start_age+factor(Gender)+factor(funding_indicator)+
                avs_net_cost+PC1+PC2+PC3+factor(gap_indicator)+GPA_trend+s_start_year
                +Average_GPA + academic_period + College,
                data = train,
                family=binomial())
glm_pred = as.numeric(predict(mod_glm, newdata = test, type="response") >= .5)

In [91]:
(glm_acc = round(sum(glm_pred == test$Y)/length(glm_pred),4))

[1] 0.7861

## Prediction Summary

In [92]:
res = data.frame(rbind(lda_acc, nnet_acc, glm_acc))
colnames(res) = 'Accuracy'
rownames(res) = c('LDA', 'Neural Network', 'GLM')
res

,Accuracy
LDA,0.7618
Neural Network,0.7147
GLM,0.7861
